In [18]:
import pandas as pd
from collections import defaultdict, Counter
import networkx as nx
import os
import re
import matplotlib.pyplot as plt
from rapidfuzz import fuzz
from itertools import combinations

In [19]:
absolute_path = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"  
scripts = "scripts\\"
board_dataframes = "board_dataframes\\"
yearly_interlocks = "yearly_interlocks\\"
final_scripts = "final_scripts\\"
normalized_dataframes = "normalized_dataframes\\"


years = ["1999", "2000", "2005", "2007", "2008", "2009", "2011", "2013", "2018"]

In [20]:
# Read in the board statistics DataFrame.
board_statistics_path = os.path.join(absolute_path, altered_dataframes, "sample_board_statistics.csv")
board_statistics_df = pd.read_csv(board_statistics_path)

# columns_to_drop = ["carnegie_id_x", "carnegie_id_y", "year_x", "year_y"]
# board_statistics_df = board_statistics_df.drop(columns=[col for col in columns_to_drop if col in board_statistics_df.columns])
# Create an empty dictionary for a global mapping: AffiliationId -> Institution (from board data)
global_mapping = {}

# Process each year to update the mapping.
for year in years:
    file_path = os.path.join(absolute_path, final_scripts, normalized_dataframes, f"{year}_boards_normalized.csv")
    board_df = pd.read_csv(file_path)
    
    # Build a mapping from AffiliationId to Institution name.
    # If an AffiliationId has multiple Institution names, pick the mode (most frequent).
    # (If mode() is empty, use the first encountered value.)
    mapping = board_df.groupby("AffiliationId")["Institution"].agg(
        lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
    ).to_dict()
    
    # Update the global mapping with the mapping from this year.
    global_mapping.update(mapping)

# Now update board_statistics_df:
# 1. For each row in board_statistics_df, if its AffiliationId exists in the mapping,
#    change its "Institution" to the corresponding value from the board data.
# 2. Remove any rows whose AffiliationId is not in the mapping.
board_statistics_df["Institution"] = board_statistics_df["AffiliationId"].map(global_mapping).fillna(board_statistics_df["Institution"])
board_statistics_df = board_statistics_df[board_statistics_df["AffiliationId"].isin(global_mapping.keys())]

# Write the updated board_statistics_df back to CSV.
output_path = os.path.join(absolute_path, final_scripts, normalized_dataframes, "sample_board_statistics_normalized.csv")
board_statistics_df.to_csv(output_path, index=False)
print(f"Updated board_statistics_df saved to: {output_path}")


Updated board_statistics_df saved to: C:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\connectedData\board_analysis\final_scripts\normalized_dataframes\sample_board_statistics_normalized.csv
